# 1. Installs, Imports and Settings

In [1]:
#!pip install spacy==3.1.1 
#restart runtime after this
#!python -m spacy download es_core_news_lg

import spacy
nlp = spacy.load("es_core_news_lg")

import csv
import pandas as pd
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

#  2. Reading files 

In [2]:
df = pd.read_csv('data/alertas.csv', sep="|")
df = df[df["Text"].notnull()]
print("Size of Data:", len(df))
print()
print("Columns : ", list(df.columns))

Size of Data: 666

Columns :  ['Filename', 'Text', 'Type', 'Year']


In [ ]:
' '.join(df["Text"]).split()

In [3]:
len(' '.join(df["Text"]).split())

4602954

In [4]:
import re
exceptions = [
    "SAT \n \nSede Central Calle 55 Nº 10 32 Of 115 \nTels 3147300 Ext 2437 Telefax 6915300 \nEmail", 
    "DE LA", 
    "NI OTRO", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "Ext 2437 Telefax 6915300", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "Sede Central Calle 55 No", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "SAT Sede Central Calle 55 Nº 10 32 Of 115 Email", 
    "SAT Sede Central Calle 55 Nº 1032 Of 115 Email", 
    "SA Email",
    "SAT INFORME DE RIESGO No",
    "DEL CONFLICTO ARMADO Sistema",
    "LA VIDA LA LIBERTAD Y",
    "DELEGADA PARA LA EVALUACIÓN DEL RIESGO POBLACIÓN CIVIL",
    "SAT Sede Central Calle 55 Nº 1032 Of 115 Tels 3147300 Ext 2437 Telefax 6915300 Email", 
    "SAT Sede Central Calle 55 Nº 10 32 Of 115 Tels 3147300 Ext 2437 Telefax 6915300 Email", 
    "INFORME DE RIESGO No", 
    "LA INTEGRIDAD FISICA DE LA POBLACIÓN", 
    "Tels 3147300 Ext 2437 Telefax", 
    "DELEGADA PARA LA EVALUACIÓN", 
    "LA POBLACIÓN CIVIL", 
    "Tels 3147300 Ext 2437 Telefax 6915300 Correo Electrónico", 
    "VILLEGAS Defensor Delegado",
    "Tels 3147300 Ext 2452 Telefax",
    "2437 Telefax",
    "CHACÓN Defensor Delegado", 
    "Tels 3147300 ext 24372464 Fax ext 2452 Bogotá",
    "Tels 3147300 Ext 2437 Telefax ext", 
    "Tels 3147300 Ext 2452 Telefax", 
    "Telefax Ext 2452 Correo Electrónico", 
    "2437 Telefax", 
    "Tels 3147300 ext 24372464 Fax ext 2452 Bogotá", 
    "2464 Fax ext 2452 Bogotá",
    "X", 
    "Inminencia alta Urgente Grado 2 Urgencia intermedia Grado 3 No", 
    "SAT Sede Central Calle 55 Nº 1032 Bloque C Tercer piso Tels 3147300 Ext 2437 Telefax", 
    "O SITUACIÓN CRÓNICA",
    "AUTORIDADES VINCULADAS AL", 
    
    
]
def cleantext1(text):
    text = text.lower()
    text = text.replace("\n", " ")
    for s in exceptions:
        text = text.replace(s.lower(), " ")
    text = re.sub(' +', ' ', text)
    return text    

df["Text"] = df["Text"].apply(cleantext1)

# 3. Functions

In [5]:
def acrfull(x):
    return ': '.join([x, spacy.explain(x)])

def extract_named_entities(text):    
    nlp.max_length = len(text) + 100
    doc = nlp(text)

    dt = pd.DataFrame(columns=['entity', 'label'])
    i = 0
    for ent in doc.ents:
        if ent.text not in exceptions:
            dt.loc[i] = [ent.text, ent.label_] 
            i = i +1
        
    dt = dt.reset_index()
    dt = dt.groupby(['entity', 'label']).agg({'index' : 'count'}).sort_values('index', ascending=False).reset_index()
    dt.columns = ['Entity', 'Type', '#Ocurrences']
    dt = dt.set_index('Entity')
    dt['Type'] = dt['Type'].apply(acrfull)
    
    dt = dt.sort_values('#Ocurrences', ascending=False) #sort before deleting duplicates
    dt = dt[~dt.index.duplicated(keep='first')]                 
    
    #recognizing handlers
    dt.loc[dt.index.str.startswith('@') , 'Type'] = 'Social Media Handler'
    
    return dt.sort_values('#Ocurrences', ascending=False)

# 4. Top 20 named entities by Year - Advertencias

In [7]:
years = list(df[df["Type"] == "advertencia"]['Year'].unique())
years = sorted(years)
years

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2018,
 2019,
 2020,
 2021,
 2022]

In [8]:
for y in years:
    dfg = df[(df["Type"] == "advertencia") & (df["Year"] == y)]
    print(y,"--", len(dfg), "documents")
    dt = extract_named_entities(' '.join(dfg['Text'])).head(20)
    display(dt)

2002 -- 27 documents


,Type,#Ocurrences
Entity,,
sat,"ORG: Companies, agencies, institutions, etc.",82
eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",55
farc,"ORG: Companies, agencies, institutions, etc.",54
medellín,"LOC: Non-GPE locations, mountain ranges, bodies of water",22
farc eln auc,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",19
antioquia,"LOC: Non-GPE locations, mountain ranges, bodies of water",15
magdalena,"LOC: Non-GPE locations, mountain ranges, bodies of water",12
farc eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",8
valledupar,"LOC: Non-GPE locations, mountain ranges, bodies of water",7


2003 -- 58 documents


,Type,#Ocurrences
Entity,,
farc,"ORG: Companies, agencies, institutions, etc.",169
sat,"ORG: Companies, agencies, institutions, etc.",120
eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",89
dc colombia,"LOC: Non-GPE locations, mountain ranges, bodies of water",51
farc eln auc,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",39
bogotá dc colombia,"LOC: Non-GPE locations, mountain ranges, bodies of water",26
antioquia,"LOC: Non-GPE locations, mountain ranges, bodies of water",23
san miguel,"LOC: Non-GPE locations, mountain ranges, bodies of water",22
bajo putumayo,"LOC: Non-GPE locations, mountain ranges, bodies of water",19


2004 -- 88 documents


...

,Type,#Ocurrences
Entity,,
ppp01,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",765
eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",522
bogotá,"LOC: Non-GPE locations, mountain ranges, bodies of water",338
medellín,"LOC: Non-GPE locations, mountain ranges, bodies of water",301
antioquia,"LOC: Non-GPE locations, mountain ranges, bodies of water",236
bajo cauca,"LOC: Non-GPE locations, mountain ranges, bodies of water",206
cauca,"LOC: Non-GPE locations, mountain ranges, bodies of water",189
farc ep,"ORG: Companies, agencies, institutions, etc.",178
ciprat,PER: Named person or family.,169


2021 -- 25 documents


,Type,#Ocurrences
Entity,,
eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",430
ppp01,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",417
farc ep,"ORG: Companies, agencies, institutions, etc.",140
puerto carreño,"LOC: Non-GPE locations, mountain ranges, bodies of water",133
ppp02,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",121
cravo norte,"LOC: Non-GPE locations, mountain ranges, bodies of water",115
arauca,"LOC: Non-GPE locations, mountain ranges, bodies of water",111
colombia,"LOC: Non-GPE locations, mountain ranges, bodies of water",107
antioquia,"LOC: Non-GPE locations, mountain ranges, bodies of water",100


2022 -- 14 documents


,Type,#Ocurrences
Entity,,
eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",345
ppp01,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",276
farc,"ORG: Companies, agencies, institutions, etc.",225
cali,"LOC: Non-GPE locations, mountain ranges, bodies of water",198
gdo,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",115
farc ep,"ORG: Companies, agencies, institutions, etc.",113
valle del cauca,"LOC: Non-GPE locations, mountain ranges, bodies of water",107
antioquia,"LOC: Non-GPE locations, mountain ranges, bodies of water",99
· bogotá,"LOC: Non-GPE locations, mountain ranges, bodies of water",90


# 5 Top 20 named entities in Seguimiento

In [9]:
for y in [2018, 2019]:
    print (y)
    dfg = df[(df["Type"] == "seguimiento") & (df["Year"] == y)]
    dt = extract_named_entities(' '.join(dfg['Text'])).head(20)
    display(dt)

2018


,Type,#Ocurrences
Entity,,
suarez,"LOC: Non-GPE locations, mountain ranges, bodies of water",28
ntc gp1000,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",22
córdoba,"LOC: Non-GPE locations, mountain ranges, bodies of water",16
cauca,"LOC: Non-GPE locations, mountain ranges, bodies of water",14
jaime orlando,PER: Named person or family.,14
lazona,"LOC: Non-GPE locations, mountain ranges, bodies of water",14
caldas,"LOC: Non-GPE locations, mountain ranges, bodies of water",12
anserma,PER: Named person or family.,12
ciprat,PER: Named person or family.,12


2019


,Type,#Ocurrences
Entity,,
colombia,"LOC: Non-GPE locations, mountain ranges, bodies of water",27
sat,"ORG: Companies, agencies, institutions, etc.",24
santander,"LOC: Non-GPE locations, mountain ranges, bodies of water",23
vulneratorias,PER: Named person or family.,22
antioquia,"LOC: Non-GPE locations, mountain ranges, bodies of water",22
eln,"MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art",9
s naciones unidas,"ORG: Companies, agencies, institutions, etc.",8
valle del cauca,"LOC: Non-GPE locations, mountain ranges, bodies of water",7
farc,"ORG: Companies, agencies, institutions, etc.",7


# 5. Exporting to html

In [10]:
!jupyter nbconvert --to html 3_NLP_Named_Entities.ipynb

[NbConvertApp] Converting notebook 3_NLP_Named_Entities.ipynb to html
[NbConvertApp] Writing 652346 bytes to 3_NLP_Named_Entities.html
